In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from plotting.constants import PROJECT_NAME_BASE, METRIC_NAME_MAP
from plotting.utils import get_runs, get_metrics_and_colors, get_metric_statistics, _get_n_sci

In [3]:
datasets = ["yolanda", "song", "benzene",
            "malonaldehyde", "acsincome", "houseelec"]
metric = "test_posterior_samples_mean_nll" # or "test_rmse"

In [4]:
def get_dataset_results(dataset, metric):
    runs = get_runs(PROJECT_NAME_BASE + dataset, mode="gp_inference")
    metrics_dict, _ = get_metrics_and_colors(runs, metric)
    statistics_dict = get_metric_statistics(metrics_dict, skip_inconsistent=False)
    size = runs[0].run.config["ntr"]
    return statistics_dict, size

In [5]:
statistics_dicts = {}
size_dicts = {}

for dataset in datasets:
    statistics_dict, size = get_dataset_results(dataset, metric)
    statistics_dicts[dataset] = statistics_dict
    size_dicts[dataset] = size

/home/pratikr/scalable_gp_inference/plotting/metric_classes.py:77: UserWarning: Not all MetricData objects have the same steps. This may lead to incorrect results. This is likely because some runs were not finished.
  warn(
/home/pratikr/scalable_gp_inference/plotting/metric_classes.py:90: UserWarning: Returning None for the mean, min, and max data.
  warn("Returning None for the mean, min, and max data.")


In [6]:
# Extract final mean for each dataset + optimizer combo
final_means = {}
for dataset, statistics_dict in statistics_dicts.items():
    final_means[dataset] = {}
    for optimizer, stats in statistics_dict.items():
        # print(f"{optimizer} finished = {stats[0].finished}")
        if stats[0] is not None and stats[0].finished:
            final_means[dataset][optimizer] = stats[0].metric_data[-1]
        else:
            print(f"Warning: {optimizer} did not finish for {dataset}")
            final_means[dataset][optimizer] = "Diverged"
            

In [7]:
# Format numbers to 3 decimal places, but leave "Diverged" as is
def format_value(val):
    if isinstance(val, (int, float, np.number)):
        return f"{val:.3f}"
    return val

In [8]:
# Convert to DataFrame
df = pd.DataFrame(final_means)
df = df.map(format_value)

# Generate LaTeX table with custom column headers including dataset sizes
latex_code = r"\begin{tabular}{ll" + "c" * len(df.columns) + "}\n"
latex_code += r"\toprule" + "\n"

# Create the header row with dataset names
header_row = r"& Dataset & "
for col in df.columns:
    header_row += f"{col} & "
header_row = header_row[:-3] + r" \\" + "\n"
latex_code += header_row

# Add the row with dataset sizes in scientific notation
size_row = r"& $n$ & "
for col in df.columns:
    size_row += f"${_get_n_sci(size_dicts.get(col, 0))}$ & "
size_row = size_row[:-3] + r" \\" + "\n"
latex_code += size_row

# Add midrule
latex_code += r"\midrule" + "\n"

# Add the data rows with multirow for metric
optimizers = df.index.tolist()
num_optimizers = len(optimizers)
metric_label = METRIC_NAME_MAP[metric]
for i, idx in enumerate(optimizers):
    if i == 0:  # First optimizer, add the multirow
        multirow_part = f"\\multirow{{{num_optimizers}}}{{*}}"
        rotatebox_part = f"\\rotatebox[origin=c]{{90}}{{{metric_label}}}"
        data_row = f"{multirow_part}{{{rotatebox_part}}} & {idx} & "
    else:  # Other optimizers
        data_row = r" & " + f"{idx} & "

    for col in df.columns:
        data_row += f"{df.loc[idx, col]} & "
    data_row = data_row[:-3] + r" \\" + "\n"
    latex_code += data_row

# Finish the table
latex_code += r"\bottomrule" + "\n"
latex_code += r"\end{tabular}"

print(latex_code)

\begin{tabular}{llcccccc}
\toprule
& Dataset & yolanda & song & benzene & malonaldehyde & acsincome & houseelec \\
& $n$ & $3.60 \cdot 10^{5}$ & $4.64 \cdot 10^{5}$ & $5.65 \cdot 10^{5}$ & $8.94 \cdot 10^{5}$ & $1.50 \cdot 10^{6}$ & $1.84 \cdot 10^{6}$ \\
\midrule
\multirow{6}{*}{\rotatebox[origin=c]{90}{Test Mean NLL}} & \texttt{ADASAP} & 1.179 & 1.121 & -2.673 & -2.259 & 1.229 & -2.346 \\
 & \texttt{ADASAP-I} & 1.196 & 1.170 & -0.217 & 0.466 & 1.235 & -2.185 \\
 & SDD-1 & 1.225 & 1.203 & 0.531 & 0.903 & 1.242 & -0.281 \\
 & SDD-10 & 1.187 & 1.149 & -0.762 & Diverged & 1.232 & -1.804 \\
 & SDD-100 & Diverged & Diverged & Diverged & Diverged & Diverged & Diverged \\
 & PCG & 1.179 & 1.121 & -0.124 & 0.925 & 1.316 & 2.674 \\
\bottomrule
\end{tabular}
